In [1]:
!pip install netCDF4

In [2]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

In [3]:
import zipfile
import os

input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# List all .nc files in the input folder
zip_files = [f for f in os.listdir(input_folder) if f.endswith('.zip')]

# Specify the target directory where you want to extract the ZIP file
extract_dir = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'  # Replace with the desired directory path

# Iterate through the .nc files and filter out the NO2 files
for zip_file in zip_files:
    zip_file_path = os.path.join(input_folder, zip_file)
    # Extract the ZIP file to the specified directory
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
       
    # Delete the ZIP file after extraction
    os.remove(zip_file_path)

print("Extraction and deletion completed.")

Extraction and deletion completed.


In [8]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)



need_csv_path = "/home/sysadm/Downloads/delhi_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: R

/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_102202/2607537207.py:37: R

In [9]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Combine_Co_csv'

# Output folder where combined CSV files will be stored
#output_folder = '/home/sysadm/Downloads/sentinal5p_Co/'

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)
        

In [10]:
file_paths_by_date

{'2023-09-12': ['/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-12T07:56:35Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-12T06:15:05Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-12T04:33:35Z.csv'],
 '2023-09-18': ['/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-18T06:01:50Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-18T09:24:48Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-18T07:43:19Z.csv'],
 '2023-09-26': ['/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-26T05:10:16Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-26T06:51:46Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-26T08:33:15Z.csv'],
 '2023-09-11': ['/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-11T04:52:42Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv/2023-09-11T06:34:11Z.csv',
  '/home/sysadm/D

In [12]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)



combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Delhi_Combine_Co_csv'# Output folder where combined CSV files will be stored
os.makedirs(combine_csv_output_dir,exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/delhi_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [13]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/Delhi_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Delhi_2023_sep_oct_csv"

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [9]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv"
need_csv_path = "/home/sysadm/Downloads/inside_wb_1632pts_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: i

/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/783112337.py:37: RuntimeWarning: i

In [10]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv'
combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/WB_Combine_Co_csv'

# Output folder where combined CSV files will be stored
#output_folder = '/home/sysadm/Downloads/sentinal5p_Co/'

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)
        

In [11]:
file_paths_by_date

{'2023-09-12': ['/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-12T07:56:35Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-12T06:15:05Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-12T04:33:35Z.csv'],
 '2023-09-18': ['/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-18T06:01:50Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-18T09:24:48Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-18T07:43:19Z.csv'],
 '2023-09-26': ['/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-26T05:10:16Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-26T06:51:46Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-26T08:33:15Z.csv'],
 '2023-09-11': ['/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-11T04:52:42Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv/2023-09-11T06:34:11Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_

In [12]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/WB_Co_all_csv'
combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/WB_Combine_Co_csv'# Output folder where combined CSV files will be stored
need_csv_path = "/home/sysadm/Downloads/inside_wb_1632pts_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [13]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/WB_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/WB_2023_sep_oct_csv"

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [23]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv"
os.makedirs(csv_output_dir,exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/karnataka_1650_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWa

/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/1339344054.py:37: RuntimeWa

In [24]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)


combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Combine_Co_csv'
os.makedirs(combine_csv_output_dir,exist_ok = True)



# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)
        

In [25]:
file_paths_by_date

{'2023-09-12': ['/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-12T07:56:35Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-12T06:15:05Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-12T04:33:35Z.csv'],
 '2023-09-18': ['/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-18T06:01:50Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-18T09:24:48Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-18T07:43:19Z.csv'],
 '2023-09-26': ['/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-26T05:10:16Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-26T06:51:46Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-26T08:33:15Z.csv'],
 '2023-09-11': ['/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/2023-09-11T04:52:42Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv/

In [26]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Combine_Co_csv'# Output folder where combined CSVfiles will be stored
os.makedirs(combine_csv_output_dir,exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/karnataka_1650_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [27]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/Karnataka_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Karnataka_2023_sep_oct_csv"
os.makedirs(final_csv_output_dir,exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [28]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv"
os.makedirs(csv_output_dir,exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/gujarat_1485_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: i

/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/267701667.py:37: RuntimeWarning: i

In [29]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)


combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Combine_Co_csv'
os.makedirs(combine_csv_output_dir,exist_ok = True)



# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)
        

In [30]:
file_paths_by_date

{'2023-09-12': ['/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-12T07:56:35Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-12T06:15:05Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-12T04:33:35Z.csv'],
 '2023-09-18': ['/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-18T06:01:50Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-18T09:24:48Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-18T07:43:19Z.csv'],
 '2023-09-26': ['/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-26T05:10:16Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-26T06:51:46Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-26T08:33:15Z.csv'],
 '2023-09-11': ['/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-11T04:52:42Z.csv',
  '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv/2023-09-11T06:34:11Z.csv',
  '/ho

In [31]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Combine_Co_csv'# Output folder where combined CSVfiles will be stored
os.makedirs(combine_csv_output_dir,exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/gujarat_1485_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [32]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/Gujrat_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Gujrat_2023_sep_oct_csv"
os.makedirs(final_csv_output_dir,exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [33]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Orissa_Co_all_csv"
os.makedirs(csv_output_dir,exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/orissa_1540_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWa

/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_12044/2448469805.py:37: RuntimeWa

In [34]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Orissa_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Orissa_Combine_Co_csv'# Output folder where combined CSVfiles will be stored
os.makedirs(combine_csv_output_dir,exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/orissa_1540_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [35]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/Orissa_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Orissa_2023_sep_oct_csv"
os.makedirs(final_csv_output_dir,exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [2]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Tamilnadu_Co_all_csv"
os.makedirs(csv_output_dir,exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/tamilnadu_1400_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWa

/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/1967089947.py:37: RuntimeWa

In [3]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Tamilnadu_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Tamilnadu_Combine_Co_csv'# Output folder where combined CSVfiles will be stored
os.makedirs(combine_csv_output_dir,exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/tamilnadu_1400_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [4]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/Tamilnadu_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Tamilnadu_2023_sep_oct_csv"
os.makedirs(final_csv_output_dir,exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [5]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Maharashtra_Co_all_csv"
os.makedirs(csv_output_dir,exist_ok = True)

need_csv_path = "/home/sysadm/Downloads/maharashtra_1696_points_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWa

/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_83606/2264416952.py:37: RuntimeWa

In [8]:
import os
import pandas as pd

# Input folder containing CSV files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/Maharashtra_Co_all_csv'
os.makedirs(input_folder,exist_ok = True)

combine_csv_output_dir = '/home/sysadm/Downloads/sentinal5p_Co/Maharashtra_Combine_Co_csv'# Output folder where combined CSVfiles will be stored
os.makedirs(combine_csv_output_dir,exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/maharashtra_1696_points_lat_lon.csv"   
# Load CSV data
input_coords = pd.read_csv(need_csv_path)

# Create a dictionary to store file paths by date
file_paths_by_date = {}

# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename.split('T')[0]
        
        # Append the file path to the corresponding date in the dictionary
        if date_part not in file_paths_by_date:
            file_paths_by_date[date_part] = []
        file_paths_by_date[date_part].append(file_path)



        
# Iterate through the files in the input folder
for date, file_paths in file_paths_by_date.items():
    
    # Create a CSV file name based on the start_time
    csv_file_name = f"{date}.csv"
    
    combined_df = None
    for file_path in file_paths:
        # Read the CSV file into a dataframe
        df = pd.read_csv(file_path)
        
        # Extract the 'Hcho' column and create a new dataframe with only that column
        co_df = df[['Co_value(mol/m^2)']]
        
        # Check if the combined_df is None, if so, initialize it with the 'Co' dataframe
        if combined_df is None:
            combined_df = co_df
        else:
            # Concatenate the 'Hcho' dataframe with the existing combined dataframe
            combined_df = pd.concat([combined_df, co_df], axis=1)
            
            # Create a CSV file name based on the start_time
            #csv_file_name = f"{date}.csv"
            
    combined_df = pd.concat([input_coords , combined_df], axis=1)

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(combine_csv_output_dir, csv_file_name)
    combined_df.to_csv(csv_file_path, index=False)


In [9]:
input_folder = "/home/sysadm/Downloads/sentinal5p_Co/Maharashtra_Combine_Co_csv"

final_csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Maharashtra_2023_sep_oct_csv"
os.makedirs(final_csv_output_dir,exist_ok = True)


# Iterate through the files in the input folder
for filename in os.listdir(input_folder):
    
    file_path = os.path.join(input_folder, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        # Extract the date part from the file name (assuming the date format in the file name)
        date_part = filename        
        data = pd.read_csv(file_path)
        # Group by 'longitude' and 'latitude', and find the maximum 'Co_value(mol/m^2)'
        result = data.groupby(['longitude', 'latitude'])['Co_value(mol/m^2)'].max().reset_index()
        # Save the DataFrame to a CSV file in the specified output directory
        csv_file_path = os.path.join(final_csv_output_dir, date_part)
        # Save the result to a new CSV file
        result.to_csv(csv_file_path, index=False)

In [4]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/Delhi_oct_Co_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)



need_csv_path = "/home/sysadm/Downloads/delhi_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)

/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: R

/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
/tmp/ipykernel_129612/4257491893.py:37: RuntimeWarning: invalid value encountered in sqrt
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))


In [ ]:
import os
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Function to process a single .nc file
def process_nc_file(nc_file_path, csv_output_dir,need_csv_path):
    # Open the NetCDF4 dataset
    fh = Dataset(nc_file_path, mode='r')

    # Get the necessary data from the NetCDF file (your existing code)
    lons = fh.groups['PRODUCT'].variables['longitude'][:][0,:,:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:][0,:,:]
    Co  = fh.groups['PRODUCT'].variables['carbonmonoxide_total_column'][0,:,:]

    # Extract unmasked values and replace masked values with NaN
    Co_array = np.ma.filled(Co, np.nan)
    
    # Load CSV data
    input_coords = pd.read_csv(need_csv_path)

    # Get latitudes and longitudes from the CSV
    input_latitudes = input_coords["latitude"]
    input_longitudes = input_coords["longitude"]

    # Prepare output DataFrame
    output_data = []

    for user_lon, user_lat in zip(input_longitudes, input_latitudes):
        # calculation to find nearest point in data to entered location (haversine formula)
        R = 6371000 # radius of the earth in meters
        lat1 = np.radians(user_lat)
        lat2 = np.radians(lats)
        delta_lat = np.radians(lats - user_lat)
        delta_lon = np.radians(lons - user_lon)
        a = (np.sin(delta_lat/2))*2 + (np.cos(lat1)) * (np.cos(lat2)) * (np.sin(delta_lon/2))*2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        d = R * c
        # gets the x, y location of the nearest point in data to entered location
        x, y = np.unravel_index(d.argmin(), d.shape)

        if not np.isnan(Co_array[x, y]):
            Co_value = Co_array[x, y]
        else:
            Co_value = np.nan

        output_data.append([user_lon, user_lat, Co_value])

    # Create a DataFrame with column names
    df = pd.DataFrame(output_data, columns=['longitude', 'latitude', 'Co_value(mol/m^2)'])

    # Get the value of the 'time_coverage_start' attribute
    start_time = fh.getncattr('time_coverage_start')

    # Create a CSV file name based on the start_time
    csv_file_name = f"{start_time}.csv"

    # Save the DataFrame to a CSV file in the specified output directory
    csv_file_path = os.path.join(csv_output_dir, csv_file_name)
    df.to_csv(csv_file_path, index=False)

    # Close the NetCDF file
    fh.close()

# Directory containing .nc files
input_folder = '/home/sysadm/Downloads/sentinal5p_Co/L2_data/'

# Directory where CSV files will be saved
csv_output_dir = "/home/sysadm/Downloads/sentinal5p_Co/WB_oct_Co_all_csv"
os.makedirs(csv_output_dir, exist_ok = True)


need_csv_path = "/home/sysadm/Downloads/inside_wb_1632pts_lat_lon.csv"

# List all .nc files in the input folder
nc_files = [f for f in os.listdir(input_folder) if f.endswith('.nc')]

# Initialize a list to store the paths of Co files
Co_file_paths = []

# Iterate through the .nc files and filter out the CO files
for nc_file in nc_files:
    if "CO" in nc_file:
        Co_file_path = os.path.join(input_folder, nc_file)
        Co_file_paths.append(Co_file_path)

# Process each .nc file and save it as a CSV file
for nc_file_path in Co_file_paths:
    #nc_file_path = os.path.join(input_folder, nc_file)
    process_nc_file(nc_file_path, csv_output_dir,need_csv_path)